In [2]:
!pip install tsfresh

  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: C:\Users\achar\AppData\Local\pip\Cache\wheels\0c\61\d2\d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import datetime
import sklearn
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute
import warnings
warnings.filterwarnings('ignore')
import datetime
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [4]:
def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [8]:
data = pd.read_csv('energydata_Transformation.csv', parse_dates=['date'])

In [9]:
weekType = pd.get_dummies(data['weekType'], prefix = 'weekType')
day_of_week = pd.get_dummies(data['day_of_week'], prefix = 'day_of_week')
#['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
#['Weekend', 'Weekday']

# Concat above dummies variable dataframe to the main dataframe
data = pd.concat((data,weekType),axis=1)
data = pd.concat((data,day_of_week),axis=1)

# Drop the WeekStatus and Day_of_week column
data = data.drop(['weekType','day_of_week', 'time'],axis=1)

In [10]:
check_outliers = ['Appliances', 'temp_kitchen', 'hum_kitchen', 'temp_living',
       'hum_living', 'temp_laundry', 'hum_laundry', 'temp_office',
       'hum_office', 'temp_bathroom', 'hum_bathroom', 'temp_building_out',
       'hum_building_out', 'temp_ironing', 'hum_ironing', 'temp_teenRoom',
       'hum_teenRoom', 'hum_parentRoom', 'Pressure', 'hum_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1']

for x in data[check_outliers]:
    data = data[np.abs(data[x]-data[x].mean()) <= (3*data[x].std())]

In [14]:
x = data.drop(['Appliances'],axis=1)
x.head()

,date,temp_kitchen,hum_kitchen,temp_living,hum_living,temp_laundry,hum_laundry,temp_office,hum_office,temp_bathroom,...,Numerical_Week,weekType_Weekday,weekType_Weekend,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,2016-01-11 17:00:00,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,0,1,0,0,1,0,0,0,0,0
1,2016-01-11 17:10:00,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,0,1,0,0,1,0,0,0,0,0
2,2016-01-11 17:20:00,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,0,1,0,0,1,0,0,0,0,0
3,2016-01-11 17:30:00,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,0,1,0,0,1,0,0,0,0,0
4,2016-01-11 17:40:00,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,0,1,0,0,1,0,0,0,0,0


In [15]:
x = pd.Series(data=data['Appliances'].values, index=data['date'])

In [17]:
x.head()

date
2016-01-11 17:00:00     90
2016-01-11 17:10:00     90
2016-01-11 17:20:00     80
2016-01-11 17:30:00     90
2016-01-11 17:40:00    100
dtype: int64

In [18]:
df = pd.DataFrame(x)
df.reset_index(inplace=True)

In [20]:
df.head()

,date,0
0,2016-01-11 17:00:00,90
1,2016-01-11 17:10:00,90
2,2016-01-11 17:20:00,80
3,2016-01-11 17:30:00,90
4,2016-01-11 17:40:00,100


In [21]:
from tsfresh.utilities.dataframe_functions import roll_time_series

In [22]:
df_shift, y = make_forecasting_frame(x, kind="price", max_timeshift=10, rolling_direction=1)
df_shift

,time,value,id,kind
159552,2016-01-11 17:00:00,90.0,2016-01-11 17:10:00,price
141820,2016-01-11 17:00:00,90.0,2016-01-11 17:20:00,price
159553,2016-01-11 17:10:00,90.0,2016-01-11 17:20:00,price
124089,2016-01-11 17:00:00,90.0,2016-01-11 17:30:00,price
141821,2016-01-11 17:10:00,90.0,2016-01-11 17:30:00,price
159554,2016-01-11 17:20:00,80.0,2016-01-11 17:30:00,price
106359,2016-01-11 17:00:00,90.0,2016-01-11 17:40:00,price
124090,2016-01-11 17:10:00,90.0,2016-01-11 17:40:00,price
141822,2016-01-11 17:20:00,80.0,2016-01-11 17:40:00,price
159555,2016-01-11 17:30:00,90.0,2016-01-11 17:40:00,price


In [23]:
X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [08:32<00:00, 13.64s/it]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


In [24]:
y.head()

date
2016-01-11 17:10:00     90
2016-01-11 17:20:00     80
2016-01-11 17:30:00     90
2016-01-11 17:40:00    100
2016-01-11 17:50:00     90
Name: value, dtype: int64

In [25]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(X)
features_filtered = select_features(X, y)

In [27]:
features_filtered.head().T

id,2016-01-11 17:10:00,2016-01-11 17:20:00,2016-01-11 17:30:00,2016-01-11 17:40:00,2016-01-11 17:50:00
variable,,,,,
value__quantile__q_0.3,90.000000,90.000000,86.000000,89.000000,90.000000
value__quantile__q_0.2,90.000000,90.000000,84.000000,86.000000,88.000000
value__quantile__q_0.4,90.000000,90.000000,88.000000,90.000000,90.000000
value__quantile__q_0.1,90.000000,90.000000,82.000000,83.000000,84.000000
value__median,90.000000,90.000000,90.000000,90.000000,90.000000
value__minimum,90.000000,90.000000,80.000000,80.000000,80.000000
value__quantile__q_0.6,90.000000,90.000000,90.000000,90.000000,90.000000
"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5",96.491415,96.491415,96.491415,96.491415,96.491415
"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5",117.925441,117.925441,117.925441,117.925441,117.925441


In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [30]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(features_filtered,y,test_size=0.3,random_state=42)

# model = RandomForestRegressor(n_estimators=150, max_features='sqrt', n_jobs=-1)  # случайный лес
models = [RandomForestRegressor()]

TestModels = pd.DataFrame()
tmp = {}

for model in models:
    # get model name
    m = str(model)
    tmp['Model'] = m[:m.index('(')]
    # fit model on training dataset
    model.fit(Xtrn, Ytrn)
    # predict consumption
    predictions = model.predict(Xtest)
    #Evaluation for Testing set
    #R2 score
    tmp['R2_Test'] = r2_score(Ytest,predictions)
    #Mean Absolute Error(MAE)
    tmp['MAE_Test']= mean_absolute_error(Ytest,predictions)
    #Mean Squared Error(MSE)
    tmp['MSE_Test']= mean_squared_error(Ytest,predictions)
    #Root Mean Squared Error (RMSE)
    tmp['RMSE_Test'] = np.sqrt(mean_squared_error(Ytest,predictions))
    #Evaluation for Training test
    predictions_trn = model.predict(Xtrn)
    #R2_Score
    tmp['R2_Train'] = r2_score(Ytrn,predictions_trn)
    #Mean Absolute Error(MAE)
    tmp['MAE_Train']= mean_absolute_error(Ytrn,predictions_trn)
    #Mean Squared Error(MSE)
    tmp['MSE_Train']= mean_squared_error(Ytrn,predictions_trn)
    #Root Mean Squared Error (RMSE)
    tmp['RMSE_Train'] = np.sqrt(mean_squared_error(Ytrn,predictions_trn))
    
    tmp['Training Score(%)'] = round(model.score(Xtrn, Ytrn) * 100,3)
    tmp['Testing Score(%)'] = round(model.score(Xtest, Ytest) * 100,3)
    
    tmp['MAPE_Test'] =  round(mean_absolute_percentage_error(Ytest,predictions))
    tmp['MAPE_Train'] =  round(mean_absolute_percentage_error(Ytrn,predictions_trn))
    # write obtained data
    TestModels = TestModels.append([tmp])

TestModels.set_index('Model', inplace=True)

print(TestModels)

                        MAE_Test  MAE_Train     MSE_Test  MSE_Train   R2_Test  \
Model                                                                           
RandomForestRegressor  25.930428  10.031897  2452.472791  409.14034  0.520435   

                       R2_Train  RMSE_Test  RMSE_Train  
Model                                                   
RandomForestRegressor  0.913507  49.522447   20.227218  
